<div style="color: white; display: fill; border-radius: 5px; background-color: #3ac9f0; font-size: 250%; font-family: serif; letter-spacing: 0.5px">
    <p style="padding: 20px; color: white; font-size: 30px">
        <span style="font-family:serif;">
                Telco Customer Churn Prediction with Business Rationale
        </span>
    </p>
</div>

<img width="600px" align="center" alt="Churn Idea" src="https://fullstar.cloudcircus.jp/dcms_media/image/column_churn.jpeg"/>

Customers are the core of businesses. Without customers, no company can survive in the fast-paced business environment. That is the reason many organisations are building relationships with their customers. As part of Customer Relationship Management, retaining existing customers not only keep their engagement but also increases their loyalty to the company.

A customer is classified as churned if one did not use the company's offering for a certain period. In a given period, the penetration of churned customers to the total number of customers is considered as customer churn rate. The churn rate is often used as a Key Performance Indicator for monitoring business performance. The lower the better customer retention (sometimes used to evaluate customer loyalty).

Companies may consider sending Electronic Direct Messages or discount vouchers to customers for avoiding the churn rate increment. Sending these marketing promotions to all the existing customers would be the best option to engage customers, and seems to predict customer churn does not have any obvious effect on it. Sometimes, however, these actions take costs. As a business strategist, balancing or increasing Return on Investment is the major task. If we could find customers who intend to stop the services, targeting customers at high risk of churn can reduce unnecessary resources on other customers while attracting customers at high risk of churn to re-engage with the company's offerings.

This notebook will implement customer churn prediction on a Telco and explain the rationale behind each step.

If you liked this notebook, consider upvoting. Your upvote does encourage us to work more on it. Also, appreciate it if you could leave a comment on the work for improvement.

## Import Dependencies
---

In [1]:
import numpy as np
import pandas as pd

import ast
from time import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm

np.random.seed(852)
pd.set_option('display.max_columns', None)

## Telco Customer Data Loading
---

In [2]:
file_name = "WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(f"../input/telco-customer-churn/{file_name}")

nrow, ncol = df.shape
print(f"File `{file_name}` has {ncol:,} columns with {nrow:,} rows.")

In [3]:
desc_dtype = df.dtypes
desc_iqr = df.describe() \
    .T[["min", "25%", "50%", "75%", "max"]] \
    .astype(str) \
    .apply(list, axis=1)
desc_uniq = df.select_dtypes(include="object") \
    .apply(pd.Series.unique)

desc = pd.concat([desc_iqr, desc_uniq])
desc = pd.concat([desc_dtype, desc], axis=1)
desc.columns = ["data type", "description"]
desc.index.name = "column name"

desc = desc.reset_index() \
    .groupby(["data type", "column name"])["description"] \
    .apply(list)

display(desc)

Before we dive into the prediction, it is worthwhile to check out the data we got from an external source.

The dataset contains 7,043 customers with 21 attributes. According to the data description, the dataset contains 4 types of information.
1. Demographic Information
2. Account Information
3. Services Information
4. Customer Status

From the above output, we noticed that there are 2 variables that have mismatched data type
* `SeniorCitizen` valued at numeric on 0 and 1
* `TotalCharges` valued as character numeric

Updating the above data type issues allows us to use data information properly. Let us do it in the next section.

## Raw Data Cleansing
---

In [4]:
check = df["TotalCharges"].str.contains(r"^(?!.\d)", regex=True)
display(df[check])

We are interested in why `TotalCharges` is stored as an object type. We found that there were some customers valued as " " in the column, which is also known as missing data. There are many approaches to impute the missing value, but we will fill it with numeric values of zero.

"_Customers who left within the last month – the column is called Churn_" quoting from the data description.

Now we know that the data was captured a month after the customer behaved. The available attribute also came from the "last" month. Interestingly, all customers valued at 0 on `tenure` and "No" in `Churn`. In other words, these customers are new to the company's offerings and did not pay any fees before. So, we should only impute `TotalCharges` with numeric values of zero based on the business content.

In [5]:
dc = "SeniorCitizen"
check = df[dc].astype(bool)
df[dc] = np.where(check, "Yes", "No")

dc = "TotalCharges"
df[dc] = pd.to_numeric(df[dc], errors="coerce").fillna(0)

## Data Integrity Check
---

Do you 100% trust the dataset given by others? We have faced many situations of data errors and misinterpretation in real life, and become a bit paranoid about others one's work. Just a few lines of code, we believe it is rewarding to undergo a data integrity check on the data assumption ~~although datasets available in Kaggle are less likely to have data inconsistency issues~~.

In [6]:
# check if NULL values present
assert df.isna().sum().sum() == 0

# check if customer is unique per record
assert df.shape[0] == df["customerID"].nunique()

# company's offerings
service_main = ["PhoneService", "InternetService",]
sub_serv_phone = ["MultipleLines",]
sub_serv_internet = [
    "OnlineSecurity", "OnlineBackup",
    "DeviceProtection", "TechSupport",
    "StreamingTV", "StreamingMovies",
]
service = service_main + sub_serv_phone + sub_serv_internet

# check if customer without services
assert sum(df[service].apply(lambda y: sum("No" in col for col in y), axis=1) == len(service)) == 0

# check if phone sub-service have phone service
assert all(df.query("PhoneService == 'No'")[sub_serv_phone] == "No phone service")

# check if internet sub-service have internet service
assert all(df.query("InternetService == 'No'")[sub_serv_internet] == "No internet service")

print("Data integrity check PASSED!")

## Exploratory Data Analysis
---

In [7]:
# df.to_excel("tcc_data_after_cleansing.xlsx", index=False)

Exploring the dataset is important to understand how data is distributed and behaved. Data visualisation is indeed a great tool to perform exploratory data analysis. Personally, creating fancy visualisation in Python is not very user-friendly and time-consuming. Rather than building charts in the notebook, please refer to the work on [Tableau Public](https://public.tableau.com/app/profile/jackcky/viz/TCC_EDA/EDA).

Key findings on Exploratory Data Analysis:
* About 27% of customers stopped using the services; implying a significant imbalance in response variable `Churn`
* Some attributes showed customers are less likely to churn, such as `Partner`, `Dependents`, `Contract` and `InternetService`
* The shorter period (in months) of service usage, the more likely to churn
* Customers used the services for a long usually have a lower `MonthlyCharges`; indicating a good measuring for customer loyalty

## Feature Engineering
---

If we have explanatory variables that are a linear combination of the response variable, we could build a quite decent machine learning model. However, we do not have such variables. We can create some additional attributes to describe more about each customer, such that the machine learning models know how to figure out which customers are likely to churn. We gained some insights from the Exploratory Data Analysis and inspired some ideas on reducing labels and making up extra variables.

Key tasks to implement
* Diminish value dimension by combining labels or columns
* Add an underscore in front of the column name to indicate a dropped column

In [8]:
dc = "customerID"
df.columns = ["_" + col if col in dc else col for col in df.columns]

dc = ["Partner", "Dependents",]
check = df[dc].apply(lambda y: "Yes" in list(y), axis=1)
df["Relationship"] = np.where(check, "Family", "Single")
df.columns = ["_" + col if col in dc else col for col in df.columns]

dc = "Contract"
check = df[dc].isin(["Month-to-month"])
df[dc] = np.where(check, "No", "Yes")

dc = "PaymentMethod"
check = df[dc].str.contains("automatic")
df[dc] = np.where(check, "Automatic", "Manual")

dc = [
    "MultipleLines", "OnlineSecurity", "OnlineBackup",
    "DeviceProtection", "TechSupport", "StreamingTV",
    "StreamingMovies",
]
check = df[dc].isin(["Yes", "No"])
df[dc] = df[dc].where(check, "No")

check = df["MonthlyCharges"] * df["tenure"] != df["TotalCharges"]
df["ContractUpdate"] = np.where(check, "Yes", "No")

In [9]:
df.sample(5)

## Data Pre-processing
---

To build machine learning models, we need to provide a well-structured data matrix to the models. The algorithms of each model are out of scope in this notebook, so we just implement the pre-processing step for feeding the learning algorithms. For the high-level idea to train a model, we train a model and evaluate the model using different portions of the same dataset.

Key tasks to implement:
1. Convert data into proper data type
2. Split data into train set and test set for building and evaluating models, respectively
3. Scale explanatory variables by their minimum and maximum value
4. Duplicate train set with an oversampling method to handle imbalance response variable

In [10]:
dc = "InternetService"
check = df[dc] == "DSL"
df["DslService"] = np.where(check, 1, 0)
check = df[dc] == "Fiber optic"
df["FiberOpticService"] = np.where(check, 1, 0)
df.columns = ["_" + col if col in dc else col for col in df.columns]

df = df.apply(
    lambda y: LabelEncoder().fit_transform(y) \
    if y.dtype == "object" else y
)

In [11]:
dc_y = ["Churn",]
dc_X = [col for col in df.columns if not (col in dc_y or col.startswith("_"))]
dc_oth = [col for col in df.columns if col not in dc_y + dc_X]

train, test = train_test_split(
    df, test_size=0.3, random_state=852, stratify=df["Churn"]
)

y_train, X_train, oth_train = \
    train[dc_y].values.reshape(-1), train[dc_X].values, train[dc_oth]
y_test, X_test, oth_test = \
    test[dc_y].values.reshape(-1), test[dc_X].values, test[dc_oth]

y, X, oth = df[dc_y].values.reshape(-1), df[dc_X].values, df[dc_oth]

In [12]:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X = scaler.transform(X)

In [13]:
sampler = SMOTE(random_state=852)
X_sm, y_sm = sampler.fit_resample(X_train, y_train)

In [14]:
nrow, ncol = X_train.shape
rv_ratio = sum(y_train == 1) / len(y_train) * 100
print(f"Train set has {ncol:,} columns with {nrow:,} rows.")
print(f"Positive rate of response variable is {rv_ratio:.2f}%.")
print("==================================================")

nrow, ncol = X_sm.shape
rv_ratio = sum(y_sm == 1) / len(y_sm) * 100
print(f"Train set (oversampling) has {ncol:,} columns with {nrow:,} rows.")
print(f"Positive rate of response variable is {rv_ratio:.2f}%.")
print("==================================================")

nrow, ncol = X_test.shape
rv_ratio = sum(y_test == 1) / len(y_test) * 100
print(f"Train set has {ncol:,} columns with {nrow:,} rows.")
print(f"Positive rate of response variable is {rv_ratio:.2f}%.")

## Machine Learning
---

In [15]:
def estimate_rv(pi_hat, threshold):
    
    return (pi_hat >= threshold).astype(int)

def optimise_threshold(model, X, y):
    pi_hat = model.predict_proba(X)[:, 1]
    thresholds = np.arange(0, 1, 0.001)
    scores = [f1_score(y, estimate_rv(pi_hat, th)) for th in thresholds]
    ix = np.argmax(scores)
    
    return thresholds[ix]

In [16]:
def score_model(model, X, y, threshold):
    pi_hat = model.predict_proba(X)[:, 1]
    y_hat = estimate_rv(pi_hat, threshold)
    tn, fp, fn, tp = confusion_matrix(y, y_hat).ravel()
    pos_hat = [tp, fp]
    
    metrics = np.array([])
    metrics = np.append(metrics, accuracy_score(y, y_hat))
    metrics = np.append(metrics, precision_score(y, y_hat, average=None))
    metrics = np.append(metrics, recall_score(y, y_hat, average=None))
    metrics = np.append(metrics, f1_score(y, y_hat, average=None))
    metrics = np.append(metrics, pos_hat)
    
    return metrics

In [17]:
def learn_params(model, X, y):
    exe_start = time()
    model.fit(X, y)
    threshold = optimise_threshold(model, X, y)
    elapsed_time = time() - exe_start
    params = model.best_params_
    params.update({"threshold": threshold})
    params.update({"time": elapsed_time})
    
    return model, params

def evaluate_model(tag, model, X_train, y_train, X_test, y_test):
    model, params = learn_params(model, X_train, y_train)
    evaluation = score_model(model, X_test, y_test, params["threshold"])
    
    idx_time = ["Time (sec)",]
    idx_params = ["Best Params",]
    idx_metrics = [
        "Accuracy",
        "Precision (0)", "Precision (1)",
        "Recall (0)", "Recall (1)",
        "F1-score (0)", "F1-score (1)",
    ]
    idx_cm = ["True Positive", "False Positive",]
    
    time_ = pd.Series(params.pop("time"), index=idx_time)
    params_ = pd.Series(str(params), index=idx_params)
    eval_ = pd.Series(evaluation, index=idx_metrics + idx_cm)
    
    output = pd.concat([time_, eval_, params_]).to_frame().T
    output.index = [tag]
    output[idx_time + idx_metrics] = output[idx_time + idx_metrics] \
        .astype(float) \
        .round(2)
    output[idx_cm] = output[idx_cm].astype(int)
    
    return output

#### Baseline Model

In [18]:
class BaseModel():
    def __init__(self):
        self.best_params_ = {}
        
        return None
    
    def fit(self, X, y):
        self.EY = np.mean(y)
        self.VY = np.var(y)
        
        self.alpha = ((1 - self.EY) / self.VY - 1 / self.EY) * self.EY ** 2
        self.beta = self.alpha * (1 / self.EY - 1)
        
        self.best_params_["alpha"] = self.alpha
        self.best_params_["beta"] = self.beta
        
        return None
    
    def predict_proba(self, X):
        np.random.seed(852)
        pi_hat = np.random.beta(self.alpha, self.beta, X.shape[0])
        pi_hat = pd.DataFrame({"0": pi_hat, "1": pi_hat}).values
        
        return pi_hat

model_bm = BaseModel()

#### Logistic Regression

In [19]:
lr_params = {
    "C": [0.001, 0.01, 0.1 , 1, 10, 100, 1000],
    "class_weight": ["auto"],
    "max_iter": [10000],
    "penalty": ["elasticnet"],
    "solver": ["saga"],
    "l1_ratio": [0, 0.2, 0.4, 0.6, 0.8, 1],
    "random_state": [852],
}

rs_lr_params = {
    "estimator": LogisticRegression(),
    "param_distributions": lr_params,
    "scoring": "roc_auc",
    "cv": 10,
    "verbose": 1,
    "n_jobs": -1,
}

model_lr = RandomizedSearchCV(**rs_lr_params)

#### Random Forest

In [20]:
rf_params = {
    "n_estimators" : [50, 100, 150, 200],
    "criterion" : ["gini", "entropy"],
    "max_depth" : [None, 1, 2, 5, 7, 12],
    "max_features": [None, "sqrt", "log2"],
    "random_state": [852],
}

rs_rf_params = {
    "estimator": RandomForestClassifier(),
    "param_distributions": rf_params,
    "scoring": "roc_auc",
    "cv": 10,
    "verbose": 1,
    "n_jobs": -1,
}

model_rf = RandomizedSearchCV(**rs_rf_params)

In [21]:
summary = pd.concat([
    evaluate_model("Random Guess", model_bm, X_train, y_train, X_test, y_test),
    evaluate_model("Logistic Regression", model_lr, X_train, y_train, X_test, y_test),
    evaluate_model("Logistic Regression (Oversampling)", model_lr, X_sm, y_sm, X_test, y_test),
    evaluate_model("Random Forest", model_rf, X_train, y_train, X_test, y_test),
    evaluate_model("Random Forest (Oversampling)", model_rf, X_sm, y_sm, X_test, y_test),
])

summary_coloured = summary.style.background_gradient(cmap='YlOrRd', axis=0, low=0.1, high=1.0)
display(summary_coloured)

From the above summary of model evaluation, the `Random Guess` model is underperforming while other models have similar performance. For imbalance in the response variable, oversampling method slightly improved `Logistic Regression (Oversampling)` in terms of F1-score. However, business users only being persuaded when they see some nice figure in performance. Among the fitted models, `Random Forest` outperformed in precision and will be selected as the final model.

<div class="alert alert-block alert-info">
    <span style="font-size:18px;">
        Thanks for your time reading through this notebook! This is our first data science project ever and may contain conceptual errors. That's why we would appreciate your comment and feedback for improvement. Upvoting this notebook would also motivate us to learn more and work more if you enjoyed this notebook.
    <\span>
<\div>

## (Optional) Understand the Service Charges
---

In [22]:
service = [
    "PhoneService",
    "MultipleLines",
    "DslService", "FiberOpticService",
    "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport",
    "StreamingTV", "StreamingMovies",
]

Y = df["MonthlyCharges"]
X = df[service]
model = sm.OLS(Y,X)
results = model.fit()

results.summary().tables[1]

## (Optional) Putting Model to Production
---

In [23]:
model_params = summary.loc["Random Forest", "Best Params"]
model_params = ast.literal_eval(model_params)
threshold = model_params.pop("threshold")

model = RandomForestClassifier(**model_params)
model.fit(X_train, y_train)
pi_hat = model.predict_proba(X_test)[:, 1]
y_hat = estimate_rv(pi_hat, threshold)

df_deck = test.assign(ChurnHat=y_hat).copy()
df_deck.head()